In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path
import colour
import matplotlib.pyplot as plt
import numpy
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from dhdrnet import image_loader as il

# Try and load and fuse some of the basic HDR Images

In [ ]:
DATA_DIR = Path("../../data/HDR+/dngs/")

num_imgs = 5
loaded_images = il.read_images(DATA_DIR, limit=num_imgs)

In [ ]:
for p, li in loaded_images.items():
    fig, ax = colour.plotting.plot_image(li.read_data())
    ax.set_title(p)
    plt.show()

## Adjust the exposure
Let's try and adjust the exposures. 
Picking 5 evenly spaced values over [0,2]

In [ ]:
multi_exposure = il.gen_multi_exposure(loaded_images)
for name, imgs in multi_exposure.items():
    for (md, img) in imgs:
        fig, ax = colour.plotting.plot_image(img)
        ax.set_title(name)
        plt.show()


In [ ]:
# # What about writing the images out to disk now?
# image_list = np.random.choice(list(DATA_DIR.glob("*.dng")), 5)
# il.write_multi_exposure(image_list)

In [ ]:
for name, exposures in multi_exposure.items():
    image_stack = colour_hdri.ImageStack()
    for image in exposures:
        image_stack.append(image)
    fused = colour_hdri.image_stack_to_radiance_image(image_stack)
    colour.plotting.plot_image(fused)